In [1]:
import pandas as pd
import datetime
from WindPy import w
import requests
from Portfolio import portfolio
import time
import matplotlib.pyplot as plt
import easytrader
import pprint

# 1. Connect to Wind and 华泰

In [ ]:
# start WInd
w.start()

In [2]:
# start 华泰
user = easytrader.use('ht_client')
user.prepare(user='666637575127', password='671109', comm_password='246810')

# 2. Get the current position

## Set up position template
## Our desired funds
### 1. 160325.SZ 华夏创业 2022/7/24
### 2. 160926.SZ 创业大成 2022/7/16
### 3. 501080.SH 科创中金 2022/7/11
### 4. 501082.SH 科创投资 2022/6/27
### 5. 506002.SH 易基科创 2022/7/28
### 6. 506003.SH 富国科创 2022/7/29
### 7. 506006.SH 添富科创 2022/7/28

In [3]:
position = {
    "160325":{
        "name":"华夏创业",
        "ticker":"160325.SZ",
        "price":w.wsq('160325.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 24, 0, 0, 0) - datetime.datetime.now()).days/365
    }, 
    "160926":{
        "name":"创业大成",
        "ticker":"160926.SZ",
        "price":w.wsq('160926.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 16, 0, 0, 0) - datetime.datetime.now()).days/365
    }, 
    "501080":{
        "name":"科创中金",
        "ticker":"501080.SH",
        "price":w.wsq('501080.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 11, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "501082":{
        "name":"科创投资",
        "ticker":"501082.SH",
        "price":w.wsq('501082.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 6, 27, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506002":{
        "name":"易基科创",
        "ticker":"506002.SH",
        "price":w.wsq('506002.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 28, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506003":{
        "name":"富国科创",
        "ticker":"506003.SH",
        "price":w.wsq('506003.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 29, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506006":{
        "name":"添富科创",
        "ticker":"506006.SH",
        "price":w.wsq('506006.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 28, 0, 0, 0) - datetime.datetime.now()).days/365
    },
}

IndexError: list index out of range

In [5]:
# Display 华泰 position
user.position

[{'证券代码': '501082',
  '证券名称': '科创投资',
  '股票余额': 12000,
  '可用余额': 0,
  '冻结数量': 0,
  '成本价': 2.134,
  '保本价': 2.135,
  '市价': 2.153,
  '盈亏比(%)': 0.86,
  '盈亏': 220.249,
  '市值': 25836.0,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '在途数量': 12000,
  'Unnamed: 14': ''},
 {'证券代码': '506002',
  '证券名称': '易基科创',
  '股票余额': 21100,
  '可用余额': 0,
  '冻结数量': 0,
  '成本价': 1.169,
  '保本价': 1.169,
  '市价': 1.174,
  '盈亏比(%)': 0.4,
  '盈亏': 98.069,
  '市值': 24771.4,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '在途数量': 21100,
  'Unnamed: 14': ''},
 {'证券代码': '506006',
  '证券名称': '添富科创',
  '股票余额': 22000,
  '可用余额': 0,
  '冻结数量': 0,
  '成本价': 1.161,
  '保本价': 1.161,
  '市价': 1.178,
  '盈亏比(%)': 1.43,
  '盈亏': 366.225,
  '市值': 25916.0,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '在途数量': 22000,
  'Unnamed: 14': ''},
 {'证券代码': '511880',
  '证券名称': '银华日利',
  '股票余额': 0,
  '可用余额': 0,
  '冻结数量': 0,
  '成本价': 0.0,
  '保本价': 0.0,
  '市价': 101.434,
  '盈亏比(%)': 0.0,
  '盈亏': 0.4,
  '市值': 0.0,
  '交易市场': '上海Ａ',
  '股东帐户': 'A360822222',
  '在途数量': 100,
 

## Read 华泰 position

In [43]:
ht_position = user.position

targets = [
    '506002',
    '501082',
    '506006',
    '160926',
    '506003',
    '160325',
    '501080'
]

for holding in ht_position:
    if holding['证券代码'] in targets:
        position[holding['证券代码']]['shares'] = holding['在途数量']

In [45]:
pprint.pprint(position)

{'160325': {'expiration': 0.9616438356164384,
            'name': '华夏创业',
            'price': 1.179,
            'shares': 0,
            'ticker': '160325.SZ'},
 '160926': {'expiration': 0.9397260273972603,
            'name': '创业大成',
            'price': 1.214,
            'shares': 20000,
            'ticker': '160926.SZ'},
 '501080': {'expiration': 0.9260273972602739,
            'name': '科创中金',
            'price': 1.726,
            'shares': 0,
            'ticker': '501080.SH'},
 '501082': {'expiration': 0.8876712328767123,
            'name': '科创投资',
            'price': 2.153,
            'shares': 12000,
            'ticker': '501082.SH'},
 '506002': {'expiration': 0.9726027397260274,
            'name': '易基科创',
            'price': 1.174,
            'shares': 21100,
            'ticker': '506002.SH'},
 '506003': {'expiration': 0.9753424657534246,
            'name': '富国科创',
            'price': 0.984,
            'shares': 0,
            'ticker': '506003.SH'},
 '506006':

## Calculate the fee

In [54]:
fee = ((ht_position[0]['市值'] - ht_position[0]['在途数量'] * ht_position[0]['成本价']) - ht_position[0]['盈亏'])/(ht_position[0]['在途数量'] * ht_position[0]['成本价']) * 100

print(fee, '%')

0.03026788503592629 %


In [ ]:
naive = portfolio(position, )

In [30]:
def naive1(interval, times, portfolio, threshold):
    
    cur_times = 0
    
    log_df = pd.DataFrame()
    
    while cur_times != times:
        
        now = datetime.datetime.now()
        print(now)
        
        naive.get_balance()
        
        # 1. create snapshot to document price and NAV
        snapshot = {now : {}}
        for ticker, info in portfolio.position.items():
            
            code = ticker.split('.')[0]
            headers = {
                'content-type': 'application/json',
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0'
            }
            response = requests.get(f'https://fundgz.1234567.com.cn/js/{code}.js', headers=headers)
            NAV = float(eval(response.text.replace('jsonpgz(', '').replace(');', ''))['gsz'])
            
            Price = w.wsq(ticker, 'rt_last').Data[0][0]
            info['price'] = Price
            
            AnnualizedRateOfReturn = ((NAV/Price)**(1/info['expiration'])-1)*100
            
            #NAV, Price, AnnualizedRateOfReturn
            snapshot[now][ticker + ' NAV'] = NAV
            snapshot[now][ticker + ' Price'] = Price
            snapshot[now][ticker + ' AnnualizedRateOfReturn'] = AnnualizedRateOfReturn
        
        # 2. Trade
        trade = 0
        for k, v in snapshot[now].items():
            print(k, v)
        
        min_benchmark = 100
        zero_holding_tickers = []
        
        for ticker, info in portfolio.position.items():
            
            if snapshot[now][ticker + ' AnnualizedRateOfReturn'] < min_benchmark and info['shares'] != 0:
                
                min_ticker = ticker
                min_benchmark = snapshot[now][ticker + ' AnnualizedRateOfReturn']
            
            if info['shares'] == 0:
                
                zero_holding_tickers.append(ticker)
        
        max_benchmark = 0
        
        print(min_ticker, snapshot[now][min_ticker + " AnnualizedRateOfReturn"])
        for zero_holding_ticker in zero_holding_tickers:
            print(zero_holding_ticker, snapshot[now][min_ticker + " AnnualizedRateOfReturn"])
            if snapshot['now'][zero_holding_ticker + " AnnualizedRateOfReturn"] > max_benchmark:
                max_benchmark = snapshot['now'][zero_holding_ticker + " AnnualizedRateOfReturn"]
                zeroHoldingTickerWithMaxNAV = zero_holding_ticker
        
        if snapshot[now][zeroHoldingTickerWithMaxNAV + ' AnnualizedRateOfReturn'] - snapshot[now][min_ticker + ' AnnualizedRateOfReturn'] >= threshold:
            
            trade = 1
            sell_cost = naive.position[min_ticker]['shares'] * naive.position[min_ticker]['price'] * fee
            naive.sell(min_ticker, naive.position[min_ticker]['shares'], sell_cost)
            
        snapshot[now]['Capital'] = naive.get_capital()
        naive.get_balance()
        buy_cost = naive.cash * fee
        buy_shares = round((naive.cash - buy_cost)/snapshot[now][zero_holding_ticker + ' price'], -3)
        naive.buy(zero_holding_ticker, buy_shares, buy_cost)
          
            
        snapshot[now]['Capital'] = naive.get_capital()
        snapshot[now]['Trade'] = trade
        log_df = log_df.append(pd.DataFrame(snapshot).T)
            
        naive.get_balance()
        
        cur_times += 1
        time.sleep(interval)
        print('--'*50)
    
    return log_df

In [31]:
log_df = naive1(60, 10, naive)

NameError: name 'naive' is not defined